In [1]:
import dropbox as db

In [23]:
dbx = db.Dropbox('3xr6gRMjL5AAAAAAAABHSTsOTUd_es-5Q6n_GPSkx790KlsCv7Nc4grrOGoyPDhz')
dbx.users_get_current_account()

FullAccount(account_id='dbid:AAAPaFbgZ900rnnN_KWiWuO_xeCLY-ftv8c', name=Name(given_name='Lab', surname='Jhe', familiar_name='Jhe Lab', display_name='Jhe Lab', abbreviated_name='JL'), email='jhelab.snu@gmail.com', email_verified=True, disabled=False, locale='ko', referral_link='https://www.dropbox.com/referrals/AABhUqxwE3HfZM4rvYYsQvzJ4lpXK8vKTzk?src=app9-6811472', is_paired=False, account_type=AccountType('pro', None), root_info=UserRootInfo(root_namespace_id='4797143216', home_namespace_id='4797143216'), profile_photo_url=None, country='KR', team=None, team_member_id=None)

In [27]:
print(dbx.files_list_folder(path="/23-215 ML AFM/Data").entries)

[FolderMetadata(name='Shear mode', id='id:ZawDs6pRfVAAAAAAAASjkQ', path_lower='/23-215 ml afm/data/shear mode', path_display='/23-215 ml afm/Data/Shear mode', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMetadata(name='Tapping mode', id='id:ZawDs6pRfVAAAAAAAASjkw', path_lower='/23-215 ml afm/data/tapping mode', path_display='/23-215 ml afm/Data/Tapping mode', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMetadata(name='2019.03.29', id='id:ZawDs6pRfVAAAAAAAATBgg', path_lower='/23-215 ml afm/data/2019.03.29', path_display='/23-215 ml afm/Data/2019.03.29', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None, property_groups=None), FolderMetadata(name='General mode', id='id:ZawDs6pRfVAAAAAAAAURcg', path_lower='/23-215 ml afm/data/general mode', path_display='/23-215 ml afm/Data/General mode', parent_shared_folder_id=None, shared_folder_id=None, sharing_info=None

In [49]:
import glob
import os
data_dir = './Data/Experiment/Tapping'
path = os.path.join(data_dir, '*', '*Approach*.'+'txt')
for file in glob.iglob(path, recursive = True):
    print(file)

./Data/Experiment/Tapping/try12_32758.9_5.4_24.1_60/191109Approach 9.txt
./Data/Experiment/Tapping/try12_32758.9_5.4_24.1_60/191109Approach 7.txt
./Data/Experiment/Tapping/try12_32758.9_5.4_24.1_60/191109Approach 8.txt
./Data/Experiment/Tapping/try12_32758.9_5.4_24.1_60/191109Approach 6.txt
./Data/Experiment/Tapping/try12_32758.9_5.4_24.1_60/191109Approach10.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach33.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach38.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach32.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach40.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach31.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach39.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach42.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach34.txt
./Data/Experiment/Tapping/try16_32760.1_5.7_23.6_50/1126Approach37.

In [33]:
import os
os.listdir(data_dir)

['Experiment']